<a href="https://colab.research.google.com/github/chrisbrighouse/Public/blob/main/ReadPDFandSummarise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install virtualenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 56.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.5/468.5 kB 35.4 MB/s eta 0:00:00


In [4]:
!virtualenv pdfSummary

created virtual environment CPython3.9.16.final.0-64 in 3141ms
  creator CPython3Posix(dest=/content/pdfSummary, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, setuptools=bundle, wheel=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==23.0.1, setuptools==67.4.0, wheel==0.38.4
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [ ]:
# De-activate the virtual env when done with it
!deactivate

In [5]:
!source pdfSummary/bin/activate

!pip install --upgrade pip
!pip install --no-cache-dir openai
!pip install pdfplumber
!pip install glob2
!pip install openai
#!pip install pypdf
#!pip install pypdf2
#!pip install pypdf3
!pip install reportlab pdfrw
!pip install reportlab

#!conda install -c conda-forge openai
#!pipenv install openai

# Authenticate to Google cloud drive
from google.colab import drive
drive.mount('/gdrive')
#%cd /gdrive

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [7]:
import requests
import os
import io
import pdfplumber
from time import time,sleep
import textwrap
import re
import glob2
import openai
import textwrap
from IPython.display import Audio
from google.colab import drive
#from PyPDF2 import PdfReader, PdfWriter
from pdfrw import PdfReader, PdfWriter

from PyPDF2.generic import RectangleObject
from io import BytesIO
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

#depracated exception
class PageSizeNotDefinedError(Exception):
    pass

# Define the path to the Google Drive top level
file_folder_path = '/gdrive/MyDrive/'

filepath="input.txt"
def open_file(filepath):
  print (f"opening file: {filepath}")
  with open(filepath, 'r', encoding='utf-8') as infile:
    return infile.read()

def output_file_replace (filepath, content):
  print (f"output file: {filepath}")
  with open(filepath, 'w', )as f:
    f. write(content)
    return f

#single file version
def convert_pdf2txt(src_file, dest_file):
  try:
    with pdfplumber.open(src_file) as pdf:
     # pdfReader = PyPDF2.PdfFileReader(pdf)
     # print(pdfReader.getPage(0))
      output = ''
      for page in pdf.pages:
        # Extract text from each page and append to output
        output += page.extract_text()

      # Write output to file in destination directory
      with open(dest_file, "w") as outfile:
            outfile.write(output)

  except pdfplumber.PDFSyntaxError as e:
    print(f"PDFSyntaxError: {file}")
  except Exception as e:
    print(f"{e}: {file}")

#folder convert version
def convert_pdf2txt_dir(src_dir, dest_dir):
    files = os.listdir(src_dir)
    files = [i for i in files if '.pdf' in i]
    for file in files:
        try:
            with pdfplumber.open(os.path.join(src_dir, file)) as pdf:
                output = ''
                for page in pdf.pages:
                    # Extract text from each page and append to output
                    output += page.extract_text()
                # Write output to file in destination directory
                with open(os.path.join(dest_dir, f"{file}.txt"), "w") as outfile:
                    outfile.write(output)
        except pdfplumber.PDFSyntaxError as e:
            print(f"PDFSyntaxError: {file}")
        except Exception as e:
            print(f"{e}: {file}")



#use file name without extension in INPUT_NAME
INPUT_NAME="Diesel Generator Set Fuel System Preventative Maintenance"
#INPUT_NAME="AO-connector"
#INPUT_NAME="iso_50001_2017"
#INPUT_NAME="Decarbonizing the oil and"
#INPUT_NAME="product-data-sheet-rosemount-3051s-series"
SUMMARY_TRANSLATION="Spanish"

SUMMARY_PROMPT_FILE="SummaryPrompt.txt"
OUTPUT_TXT_FILE=f"{INPUT_NAME}.txt"
OUTPUT_PDF_FILE=f"{INPUT_NAME}_summary.pdf"
OUTPUT_PDF_TRANS_FILE=f"{INPUT_NAME}_summary_translation.pdf"
OUTPUT_VOICE_FILE=f"{INPUT_NAME}.wav"

elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
openai.api_key = open_file (file_folder_path + "API Keys/openaikey.txt").strip()

input_file_path = f"{file_folder_path}PDFs/{INPUT_NAME}.pdf"
print (f"Input file: {input_file_path}")

summary_prompt_file_path = f"{file_folder_path}Prompts/{SUMMARY_PROMPT_FILE}"
print (f"Summary Prompt file: {summary_prompt_file_path}")

txt_input_file_path = f"{file_folder_path}Output/{INPUT_NAME}.txt"
print (f"Txt input file: {txt_input_file_path}")

output_file_path = f"{file_folder_path}Output/{OUTPUT_TXT_FILE}"
print (f"Output TXT file: {output_file_path}")

summary_pdf_path = f"{file_folder_path}Output/{OUTPUT_PDF_FILE}"
print (f"Output PDF file: {summary_pdf_path}")

summary_pdf_trans_path = f"{file_folder_path}Output/{OUTPUT_PDF_TRANS_FILE}"
print (f"Translated PDF file: {summary_pdf_trans_path}")

output_voice_path = f"{file_folder_path}Output/{OUTPUT_VOICE_FILE}"
print (f"Output file: {output_voice_path}")

#this uses chatgpt
def gpt_3 (messages):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": messages}]
  )
  text = response['choices'][0]['message']['content']
  return text



def gpt_4(message):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        prompt=message,
        max_tokens=2048,
        temperature=0.5,
        n=1,
        stop=None,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].text.strip() 

def summary_to_pdf (text, output_file, sentiment):
  
  #add a title
  text_with_title = f"Sentiment: {sentiment}\nSummary:\n{text}"

  #wrap the text
  output_text = textwrap.fill(text_with_title, 72)
  #print (f"Wrapped text:\n{output_text}")

  # create a new PDF document
  packet = io.BytesIO()
  can = canvas.Canvas(packet, pagesize=letter)

  # set the starting position for the text
  x = 72
  y = 700

    # split the wrapped text into lines and write each line to the PDF
  lines = output_text.split('\n')
  for line in lines:
      can.drawString(x, y, line)
      y -= 20  # adjust the vertical position for the next line

  can.save()

  # retrieve the PDF from the BytesIO object
  packet.seek(0)
  new_pdf = PdfReader(packet)
  output = PdfWriter()

  page = new_pdf.pages[0]
  output.addpage(page)

  with open (output_file, "wb") as f:
    output.write(f)

def translate_text(text, target_language):
    prompt = f"Translate '{text}' into {target_language}."
    response = openai.Completion.create(
        engine="text-davinci-002",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()

def analyze_sentiment(text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Sentiment analysis:\n{text}\nSentiment:",
        max_tokens=1,
        n=1,
        stop=None,
        temperature=0.5,
    )

    sentiment = response.choices[0].text.strip()
    return sentiment

def txt2speech (message):
  #print ("Elevenlabs input: ", message)
  #elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
  
  #Elevenlabs
  preview = "21m00Tcm4TlvDq8ikWAM"
  Ellie = "MF3mGyEYCl7XYWbV9V6O"
  Domi = "AZnzlk1XvdvUeBnXmlld"
  Bella = "EXAVITQu4vr4xnSDxMaL"
  Antoni = "ErXwobaYiN019PkySvjV"
  Josh = "TxGEqnHWrfWFTfGW9XjX"
  Arnold = "VR6AewLTigWG4xSOukaG"
  Adam = "pNInz6obpgDQGcFmaJgB"
  Sam = "yoZ06aMxZJJ28mfd3POQ"

  voice_id = Adam
  elevenlabs_tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
  print (elevenlabs_tts_url)
  output_file = output_voice_path

  #Request payload
  headers = {
      "xi-api-key": elapikey,
      "accept": "audio/mpeg",
      "Content-Type": "application/json"
  }

  data = {
      "text": message,
      "lang": "en_US",
  }

  #Send request to Elevenlabs TTS API
  response = requests.post(elevenlabs_tts_url, headers=headers, json=data)

  # Check response status
  if response.status_code == 200:
    #Write audio to file
    with open(output_voice_path, "wb") as f:
      f.write(response.content)

    #Display audio
    display(Audio(output_voice_path))
  else:
    print(f"Error: {response.status_code}")



if __name__ == '__main__':

  #Generate text from the pdf
  #print("Extracting text...")
  convert_pdf2txt(input_file_path, txt_input_file_path)

  #build the full prompt
  print("Generating Summary...")
  summary_prompt = open_file(summary_prompt_file_path).replace('<<INPUT>>', open_file(txt_input_file_path))
  #print(f"Summary prompt: {summary_prompt}")
  
  #Run the prompt through ChatGPT and save the output
  completion = gpt_3(summary_prompt)
  #print('\n\n', completion)

  #Add the Sentiment analysis
  print ("Adding sentiment analysis... ")
  sentiment_text=analyze_sentiment (summary_prompt)

  output_file_replace (output_file_path, completion)

  #Write the summary to a pdf
  print ("Writing English summary to pdf...")
  summary_to_pdf (completion, summary_pdf_path, sentiment_text)

  #Translate the summary
  print ("Generating " + SUMMARY_TRANSLATION + " pdf...")

  translated_summary = translate_text(completion, SUMMARY_TRANSLATION)
  #print (f"Translated Summary:\n{translated_summary}")

  #Write the summary to a pdf
  summary_to_pdf (translated_summary, summary_pdf_trans_path, sentiment_text)

  #Convert the English output to voice audio
  print (f"Generating audio summary...")
  txt2speech (completion)










opening file: /gdrive/MyDrive/API Keys/elevenlabskey.txt
opening file: /gdrive/MyDrive/API Keys/openaikey.txt
Input file: /gdrive/MyDrive/PDFs/Diesel Generator Set Fuel System Preventative Maintenance.pdf
Summary Prompt file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
Txt input file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output TXT file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output PDF file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance_summary.pdf
Translated PDF file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance_summary_translation.pdf
Output file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.wav
Generating Summary...
opening file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
opening file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Adding sentiment a

In [ ]:
import requests
import os
import io
import pdfplumber
from time import time,sleep
import textwrap
import re
import glob2
import openai
import textwrap
from IPython.display import Audio
from google.colab import drive
#from PyPDF2 import PdfReader, PdfWriter
from pdfrw import PdfReader, PdfWriter

#from PyPDF2.generic import RectangleObject
from io import BytesIO
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter

#depracated exception
#class PageSizeNotDefinedError(Exception):
#    pass

# Define the path to the Google Drive top level
file_folder_path = '/gdrive/MyDrive/'

filepath="input.txt"
def open_file(filepath):
  print (f"opening file: {filepath}")
  with open(filepath, 'r', encoding='utf-8') as infile:
    return infile.read()

def output_file_replace (filepath, content):
  print (f"output file: {filepath}")
  with open(filepath, 'w', )as f:
    f. write(content)
    return f

#single file version
def convert_pdf2txt(src_file, dest_file):
  try:
    with pdfplumber.open(src_file) as pdf:
     # pdfReader = PyPDF2.PdfFileReader(pdf)
     # print(pdfReader.getPage(0))
      output = ''
      for page in pdf.pages:
        # Extract text from each page and append to output
        output += page.extract_text()

      # Write output to file in destination directory
      with open(dest_file, "w") as outfile:
            outfile.write(output)

  except pdfplumber.PDFSyntaxError as e:
    print(f"PDFSyntaxError: {file}")
  except Exception as e:
    print(f"{e}: {file}")

#folder convert version
def convert_pdf2txt_dir(src_dir, dest_dir):
    files = os.listdir(src_dir)
    files = [i for i in files if '.pdf' in i]
    for file in files:
        try:
            with pdfplumber.open(os.path.join(src_dir, file)) as pdf:
                output = ''
                for page in pdf.pages:
                    # Extract text from each page and append to output
                    output += page.extract_text()
                # Write output to file in destination directory
                with open(os.path.join(dest_dir, f"{file}.txt"), "w") as outfile:
                    outfile.write(output)
        except pdfplumber.PDFSyntaxError as e:
            print(f"PDFSyntaxError: {file}")
        except Exception as e:
            print(f"{e}: {file}")



#use file name without extension in INPUT_NAME
INPUT_NAME="Diesel Generator Set Fuel System Preventative Maintenance"
#INPUT_NAME="AO-connector"
#INPUT_NAME="iso_50001_2017"
#INPUT_NAME="Decarbonizing the oil and"
#INPUT_NAME="product-data-sheet-rosemount-3051s-series"
SUMMARY_TRANSLATION="Spanish"

SUMMARY_PROMPT_FILE="SummaryPrompt.txt"
OUTPUT_TXT_FILE=f"{INPUT_NAME}.txt"
OUTPUT_PDF_FILE=f"{INPUT_NAME}_summary.pdf"
OUTPUT_PDF_TRANS_FILE=f"{INPUT_NAME}_summary_translation.pdf"
OUTPUT_VOICE_FILE=f"{INPUT_NAME}.wav"

elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
openai.api_key = open_file (file_folder_path + "API Keys/openaikey.txt").strip()

input_file_path = f"{file_folder_path}PDFs/{INPUT_NAME}.pdf"
print (f"Input file: {input_file_path}")

summary_prompt_file_path = f"{file_folder_path}Prompts/{SUMMARY_PROMPT_FILE}"
print (f"Summary Prompt file: {summary_prompt_file_path}")

txt_input_file_path = f"{file_folder_path}Output/{INPUT_NAME}.txt"
print (f"Txt input file: {txt_input_file_path}")

output_file_path = f"{file_folder_path}Output/{OUTPUT_TXT_FILE}"
print (f"Output TXT file: {output_file_path}")

summary_pdf_path = f"{file_folder_path}Output/{OUTPUT_PDF_FILE}"
print (f"Output PDF file: {summary_pdf_path}")

summary_pdf_trans_path = f"{file_folder_path}Output/{OUTPUT_PDF_TRANS_FILE}"
print (f"Translated PDF file: {summary_pdf_trans_path}")

output_voice_path = f"{file_folder_path}Output/{OUTPUT_VOICE_FILE}"
print (f"Output file: {output_voice_path}")

#this uses chatgpt
def gpt_3 (messages):
  response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[{"role": "user", "content": messages}]
  )
  text = response['choices'][0]['message']['content']
  return text



def gpt_4(message):
    response = openai.ChatCompletion.create(
        model="gpt-4",
        prompt=message,
        max_tokens=2048,
        temperature=0.5,
        n=1,
        stop=None,
        frequency_penalty=0,
        presence_penalty=0
    )

    return response.choices[0].text.strip() 

def summary_to_pdf (text, output_file, sentiment):
  
  #add a title
  text_with_title = f"\nSentiment: {sentiment}\nSummary:\n{text}"

  #wrap the text
  output_text = textwrap.fill(text_with_title, 72)
  #print (f"Wrapped text:\n{output_text}")

  # create a new PDF document
  packet = io.BytesIO()
  can = canvas.Canvas(packet, pagesize=letter)

  # set the starting position for the text
  x = 72
  y = 700

    # split the wrapped text into lines and write each line to the PDF
  lines = output_text.split('\n')
  for line in lines:
      can.drawString(x, y, line)
      y -= 20  # adjust the vertical position for the next line

  can.save()

  # retrieve the PDF from the BytesIO object
  packet.seek(0)
  new_pdf = PdfReader(packet)
  output = PdfWriter()

  page = new_pdf.pages[0]
  output.addpage(page)

  with open (output_file, "wb") as f:
    output.write(f)

def translate_text(text, target_language):
    prompt = f"Translate '{text}' into {target_language}."
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=prompt,
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )
    return response.choices[0].text.strip()

def analyze_sentiment(text):
    response = openai.Completion.create(
        engine="text-davinci-003",
        prompt=f"Sentiment analysis:\n{text}\nSentiment:",
        max_tokens=1024,
        n=1,
        stop=None,
        temperature=0.5,
    )

    sentiment = response.choices[0].text.strip()
    return sentiment

def txt2speech (message):
  #print ("Elevenlabs input: ", message)
  #elapikey = open_file (file_folder_path + "API Keys/elevenlabskey.txt").strip()
  
  #Elevenlabs
  preview = "21m00Tcm4TlvDq8ikWAM"
  Ellie = "MF3mGyEYCl7XYWbV9V6O"
  Domi = "AZnzlk1XvdvUeBnXmlld"
  Bella = "EXAVITQu4vr4xnSDxMaL"
  Antoni = "ErXwobaYiN019PkySvjV"
  Josh = "TxGEqnHWrfWFTfGW9XjX"
  Arnold = "VR6AewLTigWG4xSOukaG"
  Adam = "pNInz6obpgDQGcFmaJgB"
  Sam = "yoZ06aMxZJJ28mfd3POQ"

  voice_id = Adam
  elevenlabs_tts_url = f"https://api.elevenlabs.io/v1/text-to-speech/{voice_id}"
  print (elevenlabs_tts_url)
  output_file = output_voice_path

  #Request payload
  headers = {
      "xi-api-key": elapikey,
      "accept": "audio/mpeg",
      "Content-Type": "application/json"
  }

  data = {
      "text": message,
      "lang": "en_US",
  }

  #Send request to Elevenlabs TTS API
  response = requests.post(elevenlabs_tts_url, headers=headers, json=data)

  # Check response status
  if response.status_code == 200:
    #Write audio to file
    with open(output_voice_path, "wb") as f:
      f.write(response.content)

    #Display audio
    display(Audio(output_voice_path))
  else:
    print(f"Error: {response.status_code}")



if __name__ == '__main__':

  #Generate text from the pdf
  #print("Extracting text...")
  convert_pdf2txt(input_file_path, txt_input_file_path)

  #build the full prompt
  print("Generating Summary...")
  summary_prompt = open_file(summary_prompt_file_path).replace('<<INPUT>>', open_file(txt_input_file_path))
  #print(f"Summary prompt: {summary_prompt}")
  
  #Run the prompt through ChatGPT and save the output
  completion = gpt_3(summary_prompt)
  #print('\n\n', completion)

  output_file_replace (output_file_path, completion)

  #Add the Sentiment analysis
  print ("Adding sentiment analysis... ")
  sentiment_text=analyze_sentiment (open_file(txt_input_file_path))
  print (f"Sentiment:\n{sentiment_text}" )

  #Write the summary to a pdf
  print ("Writing English summary to pdf...")
  summary_to_pdf (completion, summary_pdf_path, sentiment_text)

  #Translate the summary
  print ("Generating " + SUMMARY_TRANSLATION + " pdf...")

  translated_summary = translate_text(completion, SUMMARY_TRANSLATION)
  #print (f"Translated Summary:\n{translated_summary}")

  #Write the summary to a pdf
  summary_to_pdf (translated_summary, summary_pdf_trans_path, sentiment_text)

  #Convert the English output to voice audio
  #print (f"Generating audio summary...")
  #txt2speech (completion)










opening file: /gdrive/MyDrive/API Keys/elevenlabskey.txt
opening file: /gdrive/MyDrive/API Keys/openaikey.txt
Input file: /gdrive/MyDrive/PDFs/Diesel Generator Set Fuel System Preventative Maintenance.pdf
Summary Prompt file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
Txt input file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output TXT file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
Output PDF file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance_summary.pdf
Translated PDF file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance_summary_translation.pdf
Output file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.wav
Generating Summary...
opening file: /gdrive/MyDrive/Prompts/SummaryPrompt.txt
opening file: /gdrive/MyDrive/Output/Diesel Generator Set Fuel System Preventative Maintenance.txt
output file: /gdri